In [ ]:
# Copyright (c) 2025, ETH Zurich

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.stats as stats
import scipy.ndimage
from tqdm import tqdm
import spekpy as spk
import h5py

from scipy import interpolate

from scipy.interpolate import griddata
import matplotlib

In [ ]:
rave_sim_dir = Path('../rave-sim').resolve()
simulations_dir = Path('<PATH_TO_STORE_SIMULATIONS>')
scratch_dir = simulations_dir

sys.path.insert(0, str(rave_sim_dir / "nist_lookup"))
from nist_lookup.xraydb_plugin import xray_delta_beta

In [ ]:
sys.path.insert(0, str(rave_sim_dir / "big-wave"))
import multisim
import config
import util
import propagation


In [ ]:
plt.rcParams["font.family"] = "serif"

In [ ]:
def calculate_G1_height(eng):
    # constants
    h = 6.62607004 * 10**(-34) # planck constant in mˆ2 kg / s
    c_0 = 299792458 # speed of light in m / s
    eV_to_joule = 1.602176634*10**(-19)
    N_A = 6.02214086 * 10**23 #[1/mol]
    
    lambda_ = h * c_0 / (eng*eV_to_joule)
    delta_diff = xray_delta_beta('Si', 2.34, eng)[0]
    height = np.pi  * lambda_ / (2*np.pi * delta_diff)

    return height


def signal_retrieval_least_squares(data, period=None, axis=-1):
    if axis != -1:
        data = np.moveaxis(data, axis, -1)

    nsteps = data.shape[-1]

    if period is None:
        period = nsteps

    phi = np.linspace(0, 2 * np.pi * nsteps / (period), nsteps, endpoint=False)
    M = np.c_[np.sin(phi), np.cos(phi), np.ones(nsteps)]
    res, chi2, _rank, _sing_vals = np.linalg.lstsq(M, data.reshape((-1, nsteps)).T, rcond=-1)

    res = res.T.reshape((*data.shape[:-1], -1))

    dabs = res[...,2]
    dphase = -np.arctan2(res[...,0], res[...,1])
    dvis = np.sqrt(res[...,0]**2 + res[...,1]**2) / dabs

    # normalization to the total number of counts
    dabs *= nsteps

    return dabs, dphase, dvis, np.nanmean(chi2)


def calculate_pixel_intensity(x, fringe, pxEdges, statistics = 'sum'):
    fringeStats = stats.binned_statistic(x, fringe, bins=pxEdges, statistic = statistics)
    return fringeStats.statistic

def perform_binned_signal_retrieval(x, wf, pxSize, nrSteps, plot_curve = True):
    leftSide = np.arange(0-pxSize/2, np.min(x), -pxSize)
    rightSide = np.arange(0 + pxSize/2, np.max(x), pxSize)
    pxEdges = np.concatenate([np.flip(leftSide), rightSide])
    int_px = []
    for i in range(nrSteps):
        int_px.append(calculate_pixel_intensity(x, wf[i,:], pxEdges))
    int_px = np.asarray(int_px)

    trans, phase, vis, _ = signal_retrieval_least_squares(int_px, period = nrSteps, axis = 0)
    if plot_curve:
        plot_curves(trans, phase, vis, pxEdges)
    return int_px, trans, phase, vis, pxEdges

def plot_curves(trans, phase, vis, pxEdges):

    fig, axs = plt.subplots(figsize=(15,6), sharex = True, nrows = 1, ncols = 2)
    axs[0].plot(pxEdges[1:], trans, label = 'Transmission')
    axs[0].set_title('Transmission')
    axs[1].plot(pxEdges[1:], vis, label = 'Visibility')
    axs[1].set_title('Visibility')

def perform_binning(x, wf, pxSize):
    leftSide = np.arange(0-pxSize/2, np.min(x), -pxSize)
    rightSide = np.arange(0 + pxSize/2, np.max(x), pxSize)
    pxEdges = np.concatenate([np.flip(leftSide), rightSide])
    int_px = calculate_pixel_intensity(x, wf, pxEdges, statistics = 'mean')
    return int_px
    
    
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def calculate_SNR(wavefronts, detector_x, phase_steps):
    summed_wf = np.zeros_like(wavefronts[0][0])
        
    for point in wavefronts:
        wf, x_point, eng = point
        
        summed_wf += wf

    return summed_wf

def get_subdict(dict_, idx):
    sub_dict = {}
    for key in dict_.keys():
        sub_dict[key] = dict_[key][idx]
    return sub_dict

def calc_Vis_theoretical(eng, Edes,m):
    V = 2/np.pi * np.abs(np.sin(np.pi / 2 * Edes / eng)**2 * np.sin(m * np.pi / 2 * Edes / eng))
    return V

def mu_h2o(eng):
    lambda_ = h * c_0 / (eng*eV_to_joule)
    beta = xray_delta_beta('H2O', 1.0, eng)[1]
    return 4 * np.pi / lambda_ * beta

In [ ]:
# constants
h = 6.62607004 * 10**(-34) # planck constant in mˆ2 kg / s
c_0 = 299792458 # speed of light in m / s
eV_to_joule = 1.602176634*10**(-19)
N_A = 6.02214086 * 10**23 #[1/mol]
E_des = 46000

lambda_ = h * c_0 / (E_des*eV_to_joule)
p2 = 4.2*10**(-6)
p0 = p1 = p2

Dn_3 = 3*p2**2/(2*lambda_) / 2
print(Dn_3)

z_g0 = 0.218
z_g1 = z_g0 + Dn_3
z_g2 = z_g0 + 2*Dn_3
z_detector = z_g2 + 0.01

h0 = h2 = 180e-6
h1 = 59e-6

print("Z0: ", z_g0)
print("Z1: ", z_g1)
print("Z2: ", z_g2)
print("Z Detector: ", z_detector)

print(Dn_3)

In [ ]:
N = 2**24
max_energy = 70000
dx = propagation.max_dx(z_g0, 10e-6, N, propagation.convert_energy_wavelength(max_energy))

In [ ]:
s = spk.Spek(kvp=70, dk = 0.1, th = 10) # Create a spectrum
s.multi_filter((('Be', 0.15), ('Al', 3))) # Create a spectrum
k, f = s.get_spectrum(edges=True) # Get the spectrum

energyRange = [4000, 70000]
dE = 100
filtering = 0.000

energies = np.arange(5, 70+0.1, 0.1)*1e3


tube_spectrum_txt = interpolate.interp1d(k*1e3, f, fill_value = 'extrapolate')
spec_txt = tube_spectrum_txt(energies)

with h5py.File('../spectra/spectrum_70_spekpy_filtered_3mmAl.h5', 'w') as h5:
    h5.create_dataset('pdf', data =  spec_txt/ np.sum(spec_txt))
    h5.create_dataset('energy', data = energies)

path_to_spectrum = os.path.abspath('../spectra/spectrum_70_spekpy_filtered_3mmAl.h5')

In [ ]:
N * dx

In [ ]:
h0 = 140e-6
h2  = 180e-6
h1 = 59e-6
dc_bottom = np.linspace(0.9, 0.1, 17)

In [ ]:
sim_paths = []
for k, dc_b in enumerate(dc_bottom):
    for dc_t in np.linspace(dc_b, 0.1, 17-k):
        config_dict = {
                "sim_params": {
                    "N": N,
                    "dx": dx,
                    "z_detector": z_g2 + 500e-6,
                    "detector_size": 0.003,
                    "detector_pixel_size_x": 1e-7,
                    "detector_pixel_size_y": 1.0,
                    "chunk_size": 256 * 1024 * 1024 // 16,  # use 256MB chunks
                },
                "use_disk_vector": False,
                "save_final_u_vectors": False,
                "dtype": "c8",
                "multisource": {
                    "type": "points",
                    "energy_range": [11000, 70000],
                    "x_range": [-10e-6, 10e-6],
                    "z": 0.0,
                    "nr_source_points": 500,
                    "seed": 1,
                    "spectrum": path_to_spectrum,
                },
                "elements": [
                    {
                        "type": "grating",
                        "pitch": p0,
                        "dc": [dc_t.item(), dc_b.item()],
                        "z_start": z_g0,
                        "thickness": h0,
                        "nr_steps": 80,
                        "x_positions": [0.0],
                        "substrate_thickness": 370*1e-6 - h0,
                        "mat_a": ["Si", 2.34],
                        "mat_b": ["Au", 19.32],
                        "mat_substrate": ["Si", 2.34],
                    },
                    {
                        "type": "grating",
                        "pitch": p1,
                        "dc": [0.5, 0.5],
                        "z_start": z_g1,
                        "thickness": h1,
                        "nr_steps": 10,
                        "x_positions": [0.0],
                        "substrate_thickness": 200 * 1e-6 - h1,
                        "mat_a": ["Si", 2.34],
                        "mat_b": None,
                        "mat_substrate": ["Si", 2.34],
                    },
                    {
                        "type": "grating",
                        "pitch": p2,
                        "dc": [0.5, 0.5],
                        "z_start": z_g2,
                        "thickness": h2,
                        "nr_steps": 30,
                        "x_positions": (np.arange(5) * p2/5).tolist(),
                        "substrate_thickness": 500*1e-6 - h2,
                        "mat_a": ["C5H8O2", 1.19],
                        "mat_b": ["Au", 19.32],
                        "mat_substrate": ["C", 2.26],
                    },
                ],
        }
        
        sim_path = multisim.setup_simulation(config_dict, Path("."), simulations_dir)
        sim_paths.append(sim_path)
        for i in range(500):
            os.system(f"CUDA_VISIBLE_DEVICES=0 ../rave-sim/fast-wave/build-Release/fastwave -s {i} {sim_path}")

In [ ]:
sim_paths

In [ ]:
def calculate_dictionary(list_simulation, pxSize, sample_thickness = None, mu_sample = None):

    I = []
    VIS = []
    SNR = []
    energies = []
    sourceSize = []
    transmission_lines = []
    visibility_lines = []
    phase_lines = []
    dc_top = []
    dc_bottom = []
    
    for idx, sim in tqdm(enumerate(list_simulation)):
        sim_path = Path(sim)
        try:
            config_dict = config.load(Path(sim_path / 'config.yaml'))
            sp = config_dict["sim_params"]
            detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
            pixel_rectangle = np.abs(detector_x) <= pxSize

            if config_dict["multisource"]["nr_source_points"] != 500:
                continue

            spectrum = config_dict["multisource"]["spectrum"]
            wavefronts = util.load_wavefronts_filtered(sim_path, x_range=None, energy_range=[11000, 70000])
    
            sourceSize.append(2*config_dict["multisource"]["x_range"][1])
            dc_top.append(config_dict["elements"][0]["dc"][0])
            dc_bottom.append(config_dict["elements"][0]["dc"][1])
            phase_steps = len(config_dict["elements"][-1]["x_positions"])
            # Calculate the wavefront
            summed_wf = np.zeros_like(wavefronts[0][0])
            energies = []
            for k, point in enumerate(wavefronts):
                wf, x_point, eng = point
                energies.append(eng)
                if sample_thickness:
                    summed_wf += wf * np.exp(-mu_sample(eng) * sample_thickness) 
                else:
                    summed_wf += wf
    
            # Convolve the wavefront
            convolved = []
            for i in range(phase_steps):
                convolved.append(np.convolve(summed_wf[i,:], pixel_rectangle, mode = 'same'))
            convolved = np.asarray(convolved)
            
            int_px, trans, phase, vis, pxEdges = perform_binned_signal_retrieval(detector_x, summed_wf[:,:], pxSize, phase_steps, plot_curve = False)
            #vis_noC.append(vis)
            #VIS.append(np.mean(vis[68:148]))
    
            
            #int_px, trans, phase, vis, pxEdges = perform_binned_signal_retrieval(detector_x, convolved[:,:], pxSize, phase_steps, plot_curve = False)
            #vis = (np.max(int_px, axis = 0) - np.min(int_px, axis = 0)) / (np.max(int_px, axis = 0) + np.min(int_px, axis = 0))
            transmission_lines.append(trans)
            visibility_lines.append(vis)
            phase_lines.append(phase)
            
            shape_ = trans.shape[0]
            middle_axis = int(shape_/2)
            idx_start = middle_axis - int(middle_axis / 2)
            idx_end = middle_axis + int(middle_axis / 2)

            #print(idx_start - idx_end)
            #idx_start = 5
            #idx_end = 22
            I.append(np.mean(trans[idx_start:idx_end]))
            VIS.append(np.mean(vis[idx_start:idx_end]))
            SNR.append(np.sqrt(1e5*I[-1]) * VIS[-1])

        except Exception as e:
            print(e)
            print(f"rejected {sim}")
            pass
    I = np.asarray(I)
    VIS = np.asarray(VIS)
    SNR = np.asarray(SNR)
    sourceSize = np.asarray(sourceSize)
    dc_top = np.asarray(dc_top)
    dc_bottom = np.asarray(dc_bottom)

    visibility_lines = np.asarray(visibility_lines)
    transmission_lines = np.asarray(transmission_lines)
    phase_lines = np.asarray(phase_lines)
    
    
    print(idx_start)
    print(idx_end)
    energies = np.asarray(energies)
    return {
        "I": I,
        "VIS": VIS,
        "Signal": SNR,
        "sourceSize": sourceSize,
        "visibility": visibility_lines,
        "transmission": transmission_lines,
        "phase": phase_lines,
        "dc_top": dc_top,
        "dc_bottom": dc_bottom
        #"energies": energies
        }, energies
    

In [ ]:
dict_plots, energies = calculate_dictionary(sim_paths, 75*1e-6)

In [ ]:
def plot_against_keys(dict_, axis1, axis2, scale1, scale2, c1, c2, keys = None, figsize = (30, 16), return_set = False):
    titels = {'I': 'Intensity in %',
             'VIS': 'Visibility',
             'Signal': r"$\sqrt{I} * V$ (a.u.)",
             'ANG': r"$p_2 / D$",
             'DOSE_G1': 'Intensity at G1',
             'SNR': r"$\sqrt{I} * V * \frac{D}{p_2}$",
             'weighted_SNR': "SNR for equal dose as GI-BCT",
             'normalized_dose': "weighted dose at G1"}
    

    
    if keys == None:
        keys = ['VIS', 'I', 'Signal', 'ANG', 'normalized_dose', 'SNR']
    N = len(keys)
    nrows = N//3
    ncols = N//nrows

    name_axis_1 = r"$DC_{top}$"
    name_axis_2 = r"$DC_{bottom}$"
    #if ax is None:
    fig, ax = plt.subplots(figsize = figsize, sharex = True, sharey = True, nrows = nrows, ncols = ncols)
    ax = ax.ravel()

    conts = []
    for i, k in enumerate(keys):

        
        # Determine the common color scale
        vmin = np.min(dict_[k])
        if k == 'Signal':
            max_dc1 = dict_[axis1][np.argmax(dict_[k])]
            max_dc2 = dict_[axis2][np.argmax(dict_[k])]
            
        vmax = np.max(dict_[k])
        #ax[i].scatter(x=dict_[axis1]*scale1, y=dict_[axis2]*scale2, c=dict_[k])
        ax[i].set_title(f'{titels[k]}', fontsize = 20)
        # Interpolate using three different methods and plot
        Ti = griddata((dict_[axis1]*scale1, dict_[axis2]*scale2), dict_[k], (c1, c2), method='linear')
        im = ax[i].contourf(c1, c2, Ti, levels = 50, vmin = vmin, vmax = vmax, cmap = 'inferno')
        cbar = plt.colorbar(im,  ax = ax[i])

        if k == 'Signal':
            max_dc1 = dict_[axis1][np.argmax(dict_[k])]
            max_dc2 = dict_[axis2][np.argmax(dict_[k])]
            print(max_dc1)
            print(max_dc2)
            ax[i].scatter(max_dc1, max_dc2, marker = 'x', color = 'red', s = 100, linewidths = 5)
        cbar.ax.tick_params(axis='both', which='both', labelsize=15)
        ax[i].set_xlim(dict_[axis1].min()*scale1, dict_[axis1].max()*scale1*0.99)
        ax[i].set_ylim(dict_[axis2].min()*scale2, dict_[axis2].max()*scale2*0.98)
        
        ax[i].set_xlabel(f'{axis1}', fontsize = 20)
        ax[i].set_ylabel(f'{axis2}', fontsize = 20)

        ax[i].tick_params(axis='x', labelsize=20)
        ax[i].tick_params(axis='y', labelsize=20)
        ax[i].axes.set_aspect('auto')

            
        ax[i].set_xlabel(f'{name_axis_1}', fontsize = 20)
        ax[i].set_ylabel(f'{name_axis_2}', fontsize = 20)
    fig.suptitle('Tapering G0', fontsize = 20, y = 1.05)

    #plt.savefig('Tapering_G0.png', bbox_inches = 'tight', transparent = True)
    if return_set:
        return conts, ax

In [ ]:
# Maximum intensity for no grating only silicon. Extrapolated from conducted measurements

I_top_09 = dict_plots['I'][np.argwhere(dict_plots['dc_bottom'] == dict_plots['dc_top'])]
m, b = np.polyfit(dc_bottom, I_top_09, 1)
I_max = m*1 + b

In [ ]:
# Scale intensity values into percent to 100% duty-cyle
dict_plots['I'] = dict_plots['I']/I_max * 100

In [ ]:
dc0 = np.arange(0.1, 0.9, 0.01)
dc2 = np.arange(0.1, 0.9, 0.01)

d0v, d2v = np.meshgrid(dc0, dc2)

keys = ['VIS', 'Signal', 'I']

plot_against_keys(dict_plots, 'dc_top', 'dc_bottom', 1, 1, d0v, d2v, keys, figsize = (19,5))
plt.savefig('DC_variation_G0.png', transparent=True, bbox_inches = 'tight')

In [ ]:
h0 = 140e-6
h2  = 180e-6
h1 = 59e-6
dc_bottom = np.linspace(0.9, 0.1, 17)

In [ ]:
sim_paths_bottom = []
for k, dc_b in enumerate(dc_bottom):
    for dc_t in np.linspace(dc_b, 0.1, 17-k):
        config_dict = {
                "sim_params": {
                    "N": N,
                    "dx": dx,
                    "z_detector": z_g2 + 370e-6,
                    "detector_size": 0.003,
                    "detector_pixel_size_x": 1e-7,
                    "detector_pixel_size_y": 1.0,
                    "chunk_size": 256 * 1024 * 1024 // 16,  # use 256MB chunks
                },
                "use_disk_vector": False,
                "save_final_u_vectors": False,
                "dtype": "c8",
                "multisource": {
                    "type": "points",
                    "energy_range": [11000, 70000],
                    "x_range": [-10e-6, 10e-6],
                    "z": 0.0,
                    "nr_source_points": 500,
                    "seed": 1,
                    "spectrum": path_to_spectrum,
                },
                "elements": [
                    {
                        "type": "grating",
                        "pitch": p0,
                        "dc": [0.5, 0.5],
                        "z_start": z_g0,
                        "thickness": 180e-6,
                        "nr_steps": 30,
                        "x_positions": [0.0],
                        "substrate_thickness": 500*1e-6 - 180e-6,
                        "mat_a": ["C5H8O2", 1.19],
                        "mat_b": ["Au", 19.32],
                        "mat_substrate": ["C", 2.26],
                    },
                    {
                        "type": "grating",
                        "pitch": p1,
                        "dc": [0.5, 0.5],
                        "z_start": z_g1,
                        "thickness": h1,
                        "nr_steps": 10,
                        "x_positions": [0.0],
                        "substrate_thickness": 200 * 1e-6 - h1,
                        "mat_a": ["Si", 2.34],
                        "mat_b": None,
                        "mat_substrate": ["Si", 2.34],
                    },
                    {
                        "type": "grating",
                        "pitch": p2,
                        "dc": [dc_t.item(), dc_b.item()],
                        "z_start": z_g2,
                        "thickness": h0,
                        "nr_steps": 80,
                        "x_positions": (np.arange(5) * p2/5).tolist(),
                        "substrate_thickness": 370*1e-6 - h0,
                        "mat_a": ["Si", 2.34],
                        "mat_b": ["Au", 19.32],
                        "mat_substrate": ["Si", 2.34],
                    },
                ],
        }
        
        sim_path = multisim.setup_simulation(config_dict, Path("."), simulations_dir)
        sim_paths_bottom.append(sim_path)
        for i in range(500):
            os.system(f"CUDA_VISIBLE_DEVICES=0 ../data/rave-sim/fast-wave/build-Release/fastwave -s {i} {sim_path}")

In [ ]:
def calculate_dictionary_g2(list_simulation, pxSize, sample_thickness = None, mu_sample = None):

    I = []
    VIS = []
    SNR = []
    energies = []
    sourceSize = []
    transmission_lines = []
    visibility_lines = []
    phase_lines = []
    dc_top = []
    dc_bottom = []
    
    for idx, sim in tqdm(enumerate(list_simulation)):
        sim_path = Path(sim)
        try:
            config_dict = config.load(Path(sim_path / 'config.yaml'))
            sp = config_dict["sim_params"]
            detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
            pixel_rectangle = np.abs(detector_x) <= pxSize

            if config_dict["multisource"]["nr_source_points"] != 500:
                continue

            spectrum = config_dict["multisource"]["spectrum"]
            wavefronts = util.load_wavefronts_filtered(sim_path, x_range=None, energy_range=[11000, 70000])
    
            sourceSize.append(2*config_dict["multisource"]["x_range"][1])
            dc_top.append(config_dict["elements"][2]["dc"][0])
            dc_bottom.append(config_dict["elements"][2]["dc"][1])
            phase_steps = len(config_dict["elements"][-1]["x_positions"])
            # Calculate the wavefront
            summed_wf = np.zeros_like(wavefronts[0][0])
            energies = []
            for k, point in enumerate(wavefronts):
                wf, x_point, eng = point
                energies.append(eng)
                if sample_thickness:
                    summed_wf += wf * np.exp(-mu_sample(eng) * sample_thickness) 
                else:
                    summed_wf += wf
    
            # Convolve the wavefront
            convolved = []
            for i in range(phase_steps):
                convolved.append(np.convolve(summed_wf[i,:], pixel_rectangle, mode = 'same'))
            convolved = np.asarray(convolved)
            
            int_px, trans, phase, vis, pxEdges = perform_binned_signal_retrieval(detector_x, summed_wf[:,:], pxSize, phase_steps, plot_curve = False)
            #vis_noC.append(vis)
            #VIS.append(np.mean(vis[68:148]))
    
            
            #int_px, trans, phase, vis, pxEdges = perform_binned_signal_retrieval(detector_x, convolved[:,:], pxSize, phase_steps, plot_curve = False)
            #vis = (np.max(int_px, axis = 0) - np.min(int_px, axis = 0)) / (np.max(int_px, axis = 0) + np.min(int_px, axis = 0))
            transmission_lines.append(trans)
            visibility_lines.append(vis)
            phase_lines.append(phase)
            
            shape_ = trans.shape[0]
            middle_axis = int(shape_/2)
            idx_start = middle_axis - int(middle_axis / 2)
            idx_end = middle_axis + int(middle_axis / 2)

            #print(idx_start - idx_end)
            #idx_start = 5
            #idx_end = 22
            I.append(np.mean(trans[idx_start:idx_end]))
            VIS.append(np.mean(vis[idx_start:idx_end]))
            SNR.append(np.sqrt(1e5*I[-1]) * VIS[-1])

        except Exception as e:
            print(e)
            print(f"rejected {sim}")
            pass
    I = np.asarray(I)
    VIS = np.asarray(VIS)
    SNR = np.asarray(SNR)
    sourceSize = np.asarray(sourceSize)
    dc_top = np.asarray(dc_top)
    dc_bottom = np.asarray(dc_bottom)

    visibility_lines = np.asarray(visibility_lines)
    transmission_lines = np.asarray(transmission_lines)
    phase_lines = np.asarray(phase_lines)
    
    
    print(idx_start)
    print(idx_end)
    energies = np.asarray(energies)
    return {
        "I": I,
        "VIS": VIS,
        "Signal": SNR,
        "sourceSize": sourceSize,
        "visibility": visibility_lines,
        "transmission": transmission_lines,
        "phase": phase_lines,
        "dc_top": dc_top,
        "dc_bottom": dc_bottom
        #"energies": energies
        }, energies
    

In [ ]:
dict_plots_g2, energies_g2 = calculate_dictionary_g2(sim_paths_bottom, 75*1e-6)

In [ ]:
def plot_against_keys_g2(dict_, axis1, axis2, scale1, scale2, c1, c2, keys = None, figsize = (30, 16), return_set = False):
    titels = {'I': 'Intensity in %',
             'VIS': 'Visibility',
             'Signal': r"$\sqrt{I} * V$ (a.u.)",
             'ANG': r"$p_2 / D$",
             'DOSE_G1': 'Intensity at G1',
             'SNR': r"$\sqrt{I} * V * \frac{D}{p_2}$",
             'weighted_SNR': "SNR for equal dose as GI-BCT",
             'normalized_dose': "weighted dose at G1"}
    

    
    if keys == None:
        keys = ['VIS', 'I', 'Signal', 'ANG', 'normalized_dose', 'SNR']
    N = len(keys)
    nrows = N//3
    ncols = N//nrows

    name_axis_1 = r"$DC_{top}$"
    name_axis_2 = r"$DC_{bottom}$"
    #if ax is None:
    fig, ax = plt.subplots(figsize = figsize, sharex = True, sharey = True, nrows = nrows, ncols = ncols)
    ax = ax.ravel()

    conts = []
    for i, k in enumerate(keys):

        
        # Determine the common color scale
        vmin = np.min(dict_[k])
        if k == 'Signal':
            max_dc1 = dict_[axis1][np.argmax(dict_[k])]
            max_dc2 = dict_[axis2][np.argmax(dict_[k])]
            
        vmax = np.max(dict_[k])
        #ax[i].scatter(x=dict_[axis1]*scale1, y=dict_[axis2]*scale2, c=dict_[k])
        ax[i].set_title(f'{titels[k]}', fontsize = 20)
        # Interpolate using three different methods and plot
        Ti = griddata((dict_[axis1]*scale1, dict_[axis2]*scale2), dict_[k], (c1, c2), method='linear')
        im = ax[i].contourf(c1, c2, Ti, levels = 50, vmin = vmin, vmax = vmax, cmap = 'inferno')
        cbar = plt.colorbar(im,  ax = ax[i])

        if k == 'Signal':
            max_dc1 = dict_[axis1][np.argmax(dict_[k])]
            max_dc2 = dict_[axis2][np.argmax(dict_[k])]
            print(max_dc1)
            print(max_dc2)
            ax[i].scatter(max_dc1, max_dc2, marker = 'x', color = 'red', s = 100, linewidths = 5)
        cbar.ax.tick_params(axis='both', which='both', labelsize=15)
        ax[i].set_xlim(dict_[axis1].min()*scale1, dict_[axis1].max()*scale1*0.99)
        ax[i].set_ylim(dict_[axis2].min()*scale2, dict_[axis2].max()*scale2*0.98)
        
        ax[i].set_xlabel(f'{axis1}', fontsize = 20)
        ax[i].set_ylabel(f'{axis2}', fontsize = 20)

        ax[i].tick_params(axis='x', labelsize=20)
        ax[i].tick_params(axis='y', labelsize=20)
        ax[i].axes.set_aspect('auto')

            
        ax[i].set_xlabel(f'{name_axis_1}', fontsize = 20)
        ax[i].set_ylabel(f'{name_axis_2}', fontsize = 20)

    fig.suptitle('Tapering G2', fontsize = 20, y = 1.05)

    #plt.savefig('Tapering_G2.png', bbox_inches = 'tight', transparent = True)
        

    if return_set:
        return conts, ax

In [ ]:
# Maximum intensity for no grating only silicon. Extrapolated from conducted measurements

I_top_g2 = dict_plots_g2['I'][np.argwhere(dict_plots_g2['dc_bottom'] == dict_plots_g2['dc_top'])]
m, b = np.polyfit(dc_bottom, I_top_g2, 1)
I_max_g2 = m*1 + b

In [ ]:
# Scale intensity values into percent to 100% duty-cyle
dict_plots_g2['I'] = dict_plots_g2['I']/I_max_g2 * 100

In [ ]:
dc0 = np.arange(0.1, 0.9, 0.01)
dc2 = np.arange(0.1, 0.9, 0.01)

d0v, d2v = np.meshgrid(dc0, dc2)

keys = ['VIS', 'Signal', 'I']

plot_against_keys_g2(dict_plots_g2, 'dc_top', 'dc_bottom', 1, 1, d0v, d2v, keys, figsize = (19,5))
plt.savefig('DC_variation_G2.png', transparent=True, bbox_inches = 'tight')